In [17]:
import os

In [18]:
# напишем requirements.txt
requirements = """
scikit-learn==1.7.2
joblib
fastapi
uvicorn
pydantic
numpy
"""

with open("requirements.txt", "w", encoding="utf-8") as f:
    f.write(requirements)

print("✅ requirements.txt создан:")

✅ requirements.txt создан:


In [19]:
# напишем наш мини API с эндпоинтами
main = """
from fastapi import FastAPI
from pydantic import BaseModel
import numpy as np
import os
import joblib

app = FastAPI()
VERSION = os.getenv("MODEL_VERSION", "v1.0.0")
# Загружаем модель (предварительно сохрани её через joblib.dump)
MODEL_PATH = os.getenv("MODEL_PATH", "models/model.pkl")
model = joblib.load(MODEL_PATH)

# Схема входных данных
class PredictRequest(BaseModel):
    x: list[float]

@app.get("/health")
def health():
    return {"status": "ok", "version": VERSION}

@app.post("/predict")
def predict(request: PredictRequest):
    X = np.array(request.x).reshape(1, -1)
    y_pred = model.predict(X)[0]

    return {
        "status": "ok",
        "prediction": int(y_pred),
        "version": VERSION
    }
"""

with open("app/main.py", "w", encoding="utf-8") as f:
    f.write(main)

print("✅ main.py создан:")

✅ main.py создан:


In [20]:
# создадим докер для запуска API с моделью
dockerfile_content = """
# Базовый образ
FROM python:3.10-slim

# Переменные окружения
ENV MODEL_PATH=/app/models/model.pkl \\
    MODEL_VERSION=v1.0.0

# Рабочая директория
WORKDIR /app

# Копирование зависимостей и установка
COPY requirements.txt .
RUN pip install --no-cache-dir -r requirements.txt

# Копируем весь проект
COPY . .

# Точка входа
CMD ["uvicorn", "app.main:app", "--host", "0.0.0.0", "--port", "8080"]
"""

dockerignore = """
__pycache__/
*.pyc
.git
.venv
tests/
"""

with open("Dockerfile", "w") as f:
    f.write(dockerfile_content.strip())

print("✅ Dockerfile создан:")

with open(".dockerignore", "w") as f:
    f.write(dockerignore.strip())

print("✅ .dockerignore создан:")

✅ Dockerfile создан:
✅ .dockerignore создан:


In [24]:
# реагизуем стратегию развертываения Blue-Green Deployment
blue_docker_compose_content = """
services:
  mlservice_blue:
    build: .
    environment:
      - MODEL_VERSION=v1.0.0
    ports:
      - "8080:8080"
"""

green_docker_compose_content = """
services:
  mlservice_green:
    build: .
    environment:
      - MODEL_VERSION=v1.1.0
    ports:
      - "8081:8080"
"""

with open("docker-compose.blue.yml", "w") as f:
    f.write(blue_docker_compose_content.strip())

print("✅ docker-compose.blue.yml создан:")

with open("docker-compose.green.yml", "w") as f:
    f.write(green_docker_compose_content.strip())

print("✅ docker-compose.green.yml создан:")

✅ docker-compose.blue.yml создан:
✅ docker-compose.green.yml создан:


In [26]:
nginx_conf = """
worker_processes 1;

events {
    worker_connections 1024;
}

http {
    upstream ml_backend {
        server mlservice_blue:8080 weight=8;
        server mlservice_green:8080 weight=2;
    }

    server {
        listen 80;
        location / {
            proxy_pass
            http://ml_backend;
            }
    }
}
"""

nginx_docker_compose_content = """
services:
  nginx:
    image: nginx:latest
    ports:
      - "8090:80"
    volumes:
      - ./nginx.conf:/etc/nginx/nginx.conf:ro
    depends_on:
      - mlservice_blue
      - mlservice_green
"""

with open("docker-compose.nginx.yml", "w") as f:
    f.write(nginx_docker_compose_content.strip())

print("✅ docker-compose.nginx.yml создан:")

with open("nginx.conf", "w") as f:
    f.write(nginx_conf.strip())

print("✅ nginx.conf создан:")

✅ docker-compose.nginx.yml создан:
✅ nginx.conf создан:


In [23]:
# создадим воркфлоу
os.makedirs(".github/workflows",exist_ok=True)
ci_cd = """
name: Model Deployment

on:
  push:
    branches: [ main ]

jobs:
  deploy:
    runs-on: ubuntu-latest
    steps:
      - uses: actions/checkout@v4

      - name: Build and push Docker image
        run: |
          docker build -t ghcr.io/${{ github.repository }}:${{ github.sha }} .
          echo "${{ secrets.GITHUB_TOKEN }}" | docker login ghcr.io -u ${{ github.actor }} --password-stdin
          docker push ghcr.io/${{ github.repository }}:${{ github.sha }}

      - name: Deploy via API
        run: |
          curl -X POST https://api.cloudprovider/deploy \
            -H "Authorization: Bearer ${{ secrets.CLOUD_TOKEN }}" \
            -d "image=ghcr.io/${{ github.repository }}:${{ github.sha }}"
"""

with open(".github/workflows/deploy.yml", "w") as f:
    f.write(dockerfile_content.strip())

In [ ]:
# загрузка на git
"""
git init
git remote add origin https://github.com/ElizavetaMaz/ml_ci_cd_MazurinaEA
git pull origin main --allow-unrelated-histories
"""